# 强化学习

## 定义：

强化学习过程主要包含五个元素：智能体、环境、状态、动作、奖励。某一时刻下，智能体处于某一状态，执行一个动作后，环境接收到动作，促使智能体进入下一个状态，同时反馈奖励，智能体的目的是为了最大化累积奖励，根据奖励的多少调整动作以获得更大的奖励。

## MAB问题：

* 老虎机有K个摇臂，每个摇臂以一定的概率吐出金币，且概率是未知的
* 玩家每次只能从K个摇臂中选择其中一个，且相邻两次选择或奖励没有任何关系
* 玩家的目的是通过一定的策略使自己的奖励最大，即得到更多的金币

合理的想法：对每个摇臂进行一定次数的尝试，得到每个摇臂的奖励均值，然后选择均值较大的摇臂，直到游戏结束。
* 选择目前均值最大的摇臂进行游行称为“利用”(exploitation)
* 选择其他的摇臂进行游戏称为“探索”(explore)

## 探索-利用困境:

在有限的游戏次数内获得最多的奖励。
* 仅探索：将游戏机会平均分配给每个臂
* 仅利用：将剩下的游戏机会全部选择当前已知的奖励最多的摇臂

合理的想法：将一少部分的游戏次数用于“探索”，剩下的大部分游戏次数用于“利用”。
可采用如下三种策略:
* $ \epsilon-greedy$ ：在游戏中设置一个探索率ε，以ε为概率进行探索；以概率1−ε 进行利用 **(探索完全随机)**


* softmax : **核心思想为摇臂有多大可能性是最佳摇臂** (当前时刻摇臂的收益概率。收益概率越高，选择的概率越高) **（探索依概率随机）**
$$p(a) = \frac{e^\frac{Q(a)}{\tau}}{\sum_{i=1}^{K}e^\frac{Q(i)}{\tau}}$$


* UCB : **不确定条件下的乐观主义原则**。Q(a)为当前摇臂的奖励均值,后一项衡量置信度,t代表当前游戏总次数，N(a)代表当前摇臂的操作次数 **（完全不使用随机）**


$$UCB = Q(a) + \sqrt\frac{2log(t)}{N(a)}$$
$$arm=argmax_a[UCB]$$

In [1]:
class Mutil_Armed_Bandit:
    def __init__(self, k, N):
        # 初始化 k 个 摇臂
        self.k = k
        # 初始化游戏次数
        self.N = N 
        # 初始化各摇臂期望奖励
        self.expect_reward = np.zeros(k)
        # 初始化各摇臂操作次数
        self.action_counts = np.zeros(k)
        # 初始化总奖励
        self.total_reward = 0
        # 各摇臂给出奖励的概率
        self.real_reward = [0.1, 0.2, 0.3, 0.8, 0.4]
        # 当前摇臂
        self.current_arm = 1
        
    def choose_arm(self, strategy, **kwargs):
        if strategy == "epsilon_greedy":
            if np.random.uniform() > kwargs["epsilon"]:
                arm = np.argmax(self.expect_reward)
            else:
                arm = np.random.choice(self.k)
        if strategy == "softmax":
            arm_prob = np.exp(np.array(
                self.expect_reward)/kwargs["tau"])/np.exp(np.array(self.expect_reward)/kwargs["tau"]).sum()
            arm = np.random.choice(self.k, p = arm_prob)
    
        return arm
    
    def update_arm(self, strategy, **kwargs):
        for i in range(self.N):
            if strategy == "epsilon_greedy":
                arm = self.choose_arm("epsilon_greedy", epsilon=kwargs["epsilon"])
            if strategy == "softmax":
                arm = self.choose_arm("softmax", tau = kwargs["tau"])
       
            
            self.current_arm = arm
            # 所选择的摇臂反馈的奖励
            arm_reward = np.random.binomial(1, self.real_reward[self.current_arm],1)
            # 更新总奖励
            self.total_reward += arm_reward
            # 更新每个摇臂的期望奖励
            self.expect_reward[self.current_arm] = (self.expect_reward[
                self.current_arm] * self.action_counts[self.current_arm] + arm_reward)/(
                self.action_counts[self.current_arm] + 1)
            # 更新每个摇臂的操作次数
            self.action_counts[self.current_arm] += 1
        return self.total_reward, self.expect_reward, self.action_counts
    
    def ucb(self):
        for i in range(self.N):
            max_upper_bound = 0
            for j in range(self.k):
                if (self.action_counts[j] > 0): 
                    # 计算探索程度的大小
                    delta_i = np.sqrt(2 * np.log(i + 1) / self.action_counts[j])
                    # 计算UCB值
                    upper_bound = self.expect_reward[j] + delta_i
                else:
                    # 初始化UCB值
                    upper_bound = 1e400
                if upper_bound > max_upper_bound:
                    # 更新UCB值
                    max_upper_bound = upper_bound
                    # 选择最佳摇臂
                    arm = j   
            self.current_arm = arm
            # 所选择的摇臂反馈的奖励
            arm_reward = np.random.binomial(1, self.real_reward[self.current_arm],1)
            # 更新总奖励
            self.total_reward += arm_reward
            # 更新每个摇臂的期望奖励
            self.expect_reward[self.current_arm] = (self.expect_reward[self.current_arm] * self.action_counts[
                self.current_arm] + arm_reward)/(self.action_counts[self.current_arm] + 1)
            # 更新每个摇臂的操作次数
            self.action_counts[self.current_arm] += 1
        return self.total_reward, self.expect_reward, self.action_counts
    
    def choose_ucb_arm(self, iters):
        if iters < self.k:
            return iters
        else:
            for k in range(self.k):
                upper_bound = np.sqrt(2 * np.log(k+1) / self.action_counts[k])
                self.expect_reward[k] += upper_bound
                arm = np.argmax(self.expect_reward)
                return arm
    
    def excecute_ucb(self):
        for i in range(self.N):
            arm = self.choose_ucb_arm(i)
            self.current_arm = arm
            arm_reward = np.random.binomial(1, self.real_reward[self.current_arm],1)
            # 更新总奖励
            self.total_reward += arm_reward
            # 更新每个摇臂的期望奖励
            self.expect_reward[self.current_arm] = (self.expect_reward[self.current_arm] * self.action_counts[
                self.current_arm] + arm_reward)/(self.action_counts[self.current_arm] + 1)
            # 更新每个摇臂的操作次数
            self.action_counts[self.current_arm] += 1
        return self.total_reward, self.expect_reward, self.action_counts

In [2]:
# epsilon_greedy
mab = Mutil_Armed_Bandit(5,500)
total_reward,expect_reward,action_counts = mab.update_arm(strategy = "epsilon_greedy",epsilon = 0.5)
total_reward,expect_reward,action_counts

(array([275]),
 array([0.125     , 0.28125   , 0.20754717, 0.76094276, 0.32608696]),
 array([ 40.,  64.,  53., 297.,  46.]))

In [3]:
# softmax
mab = Mutil_Armed_Bandit(5,500)
total_reward,expect_reward,action_counts = mab.update_arm(strategy = "softmax",tau = 0.3)
total_reward,expect_reward,action_counts

(array([301]),
 array([0.12      , 0.12903226, 0.28571429, 0.81355932, 0.4       ]),
 array([ 25.,  31.,  49., 295., 100.]))

In [4]:
# ucb
mab = Mutil_Armed_Bandit(5,500)
mab.ucb()

(array([378]),
 array([0.07142857, 0.27272727, 0.21052632, 0.85714286, 0.40625   ]),
 array([ 14.,  22.,  19., 413.,  32.]))